In [10]:
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

class KakaoRouteFinder:
    def __init__(self):
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    def find_routes(self, origin, dest):
        print(origin, dest)

        base_url = f"https://map.kakao.com/?map_type=TYPE_MAP&target=bike&sName={origin}&eName={dest}"
        self.driver.get(base_url)
        time.sleep(0.2)

        self.driver.find_element(by=By.CSS_SELECTOR, value="body").click()
        time.sleep(0.5)
        
        try:
            dimmed_layer_selector = "#dimmedLayer"
            self.driver.find_element(
                by=By.CSS_SELECTOR,
            value=dimmed_layer_selector
            ).click()
        except ElementNotInteractableException:
            pass

        def fetch_bike():
            print(" > bike")
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(1)").click()        
            time.sleep(0.2)
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(2)").click()        
            time.sleep(0.2)
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(3)").click()        
            time.sleep(0.2)

            element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView")
            return element.get_attribute('innerHTML')
        
        def fetch_walk():
            print(" > walk")
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#walktab").click()
            time.sleep(0.5)

            element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.WalkRouteResultView")
            return element.get_attribute('innerHTML')
            
        def fetch_transit():
            print(" > transit")
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#transittab").click()
            time.sleep(0.5)

            element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"ul.TransitTotalPanel")
            return element.get_attribute('innerHTML')

        try:
            return {
                'bike': fetch_bike(),
                'walk': fetch_walk(),
                'transit': fetch_transit()
            }
        except:
            print('error')

    def __del__(self):
        self.driver.close()


In [11]:
routes = KakaoRouteFinder().find_routes('대한건축학회', '서울대학교 자하연')



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\lucet\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


대한건축학회 서울대학교 자하연
 > bike
 > walk
 > transit


### Bike

In [12]:
soup = BeautifulSoup(routes['bike'], 'html.parser')
for route in (soup.find_all('li', {"class": "BikeRouteItem"})):
    res = {
        "mode": route.find('span', {"class": "mode"}).text,
        "time": route.find('span', {"class": "time"}).text.strip(),
        "distance": route.find('span', {"class": "distance"}).text,
        "altitude": route.find('span', {"class": "altitude"}).text,
        "calories": route.find('span', {"class": "calories"}).text,
    }
    print(res)

{'mode': '자전거 도로우선', 'time': '39분', 'distance': '6.4km', 'altitude': '고도 최저 17m, 최고 128m', 'calories': '234kcal'}
{'mode': '최단거리', 'time': '38분', 'distance': '6.1km', 'altitude': '고도 최저 17m, 최고 135m', 'calories': '228kcal'}
{'mode': '편안한길', 'time': '39분', 'distance': '6.4km', 'altitude': '고도 최저 17m, 최고 128m', 'calories': '234kcal'}


### Transit

In [13]:
soup = BeautifulSoup(routes['transit'], 'html.parser')
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    res = {
        "time": route.find('span', {"class": "time"}).text.strip(),
        "info": route.find('span', {"class": "walkTime"}).get('title'),
    }
    print(res)

{'time': '35분', 'info': '도보18분 | 환승1회 | 요금 1,250원 | 7.6km'}
{'time': '41분', 'info': '도보13분 | 환승1회 | 요금 1,200원 | 7.3km'}
{'time': '39분', 'info': '도보19분 | 환승1회 | 요금 1,250원 | 8.2km'}
{'time': '40분', 'info': '도보20분 | 환승1회 | 요금 1,250원 | 7.8km'}
{'time': '43분', 'info': '도보12분 | 환승1회 | 요금 1,200원 | 7.5km'}
{'time': '39분', 'info': '도보24분 | 환승1회 | 요금 1,250원 | 7.5km'}
{'time': '46분', 'info': '도보14분 | 환승1회 | 요금 1,200원 | 8.1km'}
{'time': '42분', 'info': '도보14분 | 환승1회 | 요금 1,200원 | 7.3km'}
{'time': '44분', 'info': '도보17분 | 환승1회 | 요금 1,200원 | 7.2km'}
{'time': '34분', 'info': '도보15분 | 환승2회 | 요금 1,250원 | 7.8km'}


### Walk

In [14]:
soup = BeautifulSoup(routes['walk'], 'html.parser')
for route in (soup.find_all('li', {"class": "WalkRouteItem"})):
    res = {
        "mode": route.find('span', {"class": "mode"}).text,
        "time": route.find('span', {"class": "time"}).text.strip(),
        "info": route.find('div', {"class": "info"}).text.strip(),
    }
    print(res)

{'mode': '큰길우선', 'time': '1시간 34분', 'info': '계단 1회\n281kcal소모'}
{'mode': '최단거리', 'time': '1시간 26분', 'info': '계단 1회\n257kcal소모'}
{'mode': '편안한길', 'time': '1시간 35분', 'info': '285kcal소모'}
